
<b><center><h1>DESAFIO N°1 - 8 SEMANAS DE SQL</h1></center></b>



# **IMPORTAR LIBRERIAS NECESARIAS**

In [28]:
import sqlite3
import pandas as pd
import numpy as np

# **GENERACIÓN BASE DE DATOS**

In [29]:
# Generando una conexción a una nueva base de datos

connection = sqlite3.connect('test.db')

# Generando el objeto cursos para poder interactuar con la base de datos

cursor = connection.cursor()

# **GENERACIÓN DE LAS TABLAS**

## **TABLA SALES**

In [30]:

# Generando la tabla sales
cursor.execute("""
  CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
)

""")

# Generando el commit para actualizar
connection.commit()

In [31]:
# Comprobando la existencia de la tabla
pd.read_sql_query("SELECT * FROM sales", connection)

,customer_id,order_date,product_id


In [32]:
# Generando una lista con los valores para poblar la tabla
fill_list_sales = [
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3')
]

# Utilizando executemany para poder ejecutar de manera rápida y eficiente el poblado de la tabla
cursor.executemany(
    """INSERT INTO sales (customer_id, order_date, product_id) VALUES (?, ?, ?)""", fill_list_sales)

# Actualizando los datos
connection.commit()

## **TABLA MENU**

In [33]:
# Generación Tabla Menu
cursor.execute("""
                    CREATE TABLE IF NOT EXISTS menu(
                    product_id INTEGER, product_name VARCHAR(5), price INTEGER)""")

connection.commit()


# Generando una lista con los valores para poblar la tabla
fill_list_menu = [
    ('1', 'sushi', '10'),
    ('2', 'curry', '15'),
    ('3', 'ramen', '12')
]

# Utilizando executemany para poder ejecutar de manera rápida y eficiente el poblado de la tabla
cursor.executemany(
    "INSERT INTO menu (product_id, product_name, price) VALUES (?, ?, ?)", fill_list_menu)

# Actualizando los datos
connection.commit()




## **TABLA MEMBERS**

In [34]:
# Creando tabla members

cursor.execute("""
  CREATE TABLE IF NOT EXISTS members (
  "customer_id" VARCHAR(1),
  "join_date" DATE
)
""")


# Generando la actualización
connection.commit()


# Generando una lista con los valores para poblar la tabla
fill_list_members = [
  ('A', '2021-01-07'),
  ('B', '2021-01-09')
]


# Utilizando executemany para poder ejecutar de manera rápida y eficiente el poblado de la tabla
cursor.executemany(
    """
    INSERT INTO members (customer_id, join_date) VALUES (?, ?)
    """, fill_list_members
)

# Actualizando los datos
connection.commit()

# **Revisando las tablas**

In [35]:
pd.read_sql_query("SELECT * FROM sales", connection)

,customer_id,order_date,product_id
0,A,2021-01-01,1
1,A,2021-01-01,2
2,A,2021-01-07,2
3,A,2021-01-10,3
4,A,2021-01-11,3
5,A,2021-01-11,3
6,B,2021-01-01,2
7,B,2021-01-02,2
8,B,2021-01-04,1
9,B,2021-01-11,1


In [36]:
pd.read_sql_query("SELECT * FROM menu", connection)

,product_id,product_name,price
0,1,sushi,10
1,2,curry,15
2,3,ramen,12


In [37]:
pd.read_sql_query("SELECT * FROM members", connection)

,customer_id,join_date
0,A,2021-01-07
1,B,2021-01-09


## Entendiendo las relaciones de las tablas

**Sales** se relaciona con **menu** con `product_id` que corresponde al id de cada producto. Por lo que su relación es de **n:1**.


Además **Sales** tambien se relaciona con la tabla **members** por la clave foreana `customer id` en donde la relación es **n:1.**



---



/* --------------------
   Case Study Questions
   --------------------*/

**-- 1. What is the total amount each customer spent at the restaurant?**


-- Cantidad total de cada usario gastado en el restaurant

-- customer_id, total_spend

In [45]:
query ="""
      SELECT
        s.customer_id,
        SUM(m.price) AS total_gastado
       FROM
            sales s
      LEFT JOIN menu m
        ON s.product_id = m.product_id
      GROUP BY s.customer_id
"""

pd.read_sql_query(query, connection)


,customer_id,total_gastado
0,A,76
1,B,74
2,C,36


**Explicación**: Necesitamos obtener la cantidad gastada por cada usuario. En nuestra tabla sales tenemos todas las compras realizadas y ademas el id de cada producto.

  Tenemos:

    - El registro de cada compra
  
  Nos falta:

    - El precio de cada objeto comprado


Para poder obtener los precios de cada compra necesitamos hacer un join entre **sales** y **menu** que es donde se encuentran los valores de cada producto comprado en la tabla sales

**-- 2. How many days has each customer visited the restaurant?**


-- Identificar que tabla tiene el dato de visita al restaurant

-- Contar la cantidad de dias segun cada usuario

In [47]:

query = """

  SELECT
    customer_id,
    COUNT(DISTINCT order_date) AS days_visited
  FROM
    Sales
  GROUP BY
    customer_id


"""
pd.read_sql_query(query, connection)

,customer_id,days_visited
0,A,4
1,B,6
2,C,2


**Explicación**: Una vez identificado que la tabla **sales** tiene la información de cada compra del usuario además de la **fecha en la cual realizó la compra** el problema se simplifica.
Agrupar por usuario y **contar cada fecha unica como 1 día** que ha sido visitado

**-- 3. What was the first item from the menu purchased by each customer?**


In [52]:
query ="""
  -- Tabla temporal para rankear las compras de cada usuario

  WITH primera_orden AS (

    SELECT
      s.customer_id,
      m.product_name,
      s.order_date,
      DENSE_RANK() OVER (PARTITION BY s.customer_id ORDER BY s.order_date ASC) AS ranking
    FROM
      sales s
    LEFT JOIN menu m
      ON s.product_id = m.product_id
  )

  -- Tabla final con el primer producto comprado por cada usuario

  SELECT
    customer_id,
    product_name
  FROM
    primera_orden
  WHERE
    ranking = 1


"""

pd.read_sql_query(query, connection)

,customer_id,product_name
0,A,sushi
1,A,curry
2,B,curry
3,C,ramen
4,C,ramen
